## Cloud ML Engineで 2層CNNを学習

### 事前準備
- ML Engine APIを有効にします  
  https://console.cloud.google.com/apis/api/ml.googleapis.com/overview
- cloudml-magicエクステンションをインストールします  
  次のコードセル  **!pip install cloudmlengine**  を実行し、  
  ノートブックのセッションをリセットします

In [ ]:
!pip install cloudmlmagic

ノートブックのセッションをリセットしてください  
以下のセルをリセット後に実行します

In [ ]:
# エクステンションをロードします
%load_ext cloudmlmagic

プロジェクトID、GCSバケットを設定します
<font color="red">**PROJECTID**, **BUCKET**を書き換えてください</font>

In [ ]:
%ml_init -projectId PROJECTID -bucket BUCKET -region asia-east1 -scaleTier BASIC_GPU

次のコードはML Engineで実行されます  
通常数分で終わりますが、まれにリソースが確保できずエラーになる場合があります  
その際は時間をおいて再実行してください

In [ ]:
%%ml_run cloud
# このセルのコードはML Engineで実行されます

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/")

batch_size = 50

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    y = tf.placeholder(tf.float32, [None, ], name='y')

    X_image = tf.reshape(X, [-1, 28, 28, 1])

    # 畳み込み層1層目
    conv1 = tf.layers.conv2d(
        inputs=X_image,
        filters=32,
        kernel_size=[5, 5],
        padding="SAME",
        activation=tf.nn.relu)

    # プーリング層1層目
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # 畳み込み層1層目
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="SAME",
        activation=tf.nn.relu)

    # プーリング層2層目
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # 全結合層
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(
        inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # ドロップアウト層
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.5, training=True)

    # 出力層
    logits = tf.layers.dense(inputs=dropout, units=10, name='output')
    predict = tf.argmax(logits, 1)

    # 損失
    with tf.name_scope('calc_loss'):
        onehot_labels = tf.one_hot(indices=tf.cast(y, tf.int32), depth=10)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=onehot_labels, logits=logits, name='xentropy')
        loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

    # 損失の最適化
    train_op = tf.train.AdamOptimizer(0.0001).minimize(loss)

    # 正解率算出
    with tf.name_scope('calc_accuracy'):
        correct_prediction = tf.equal(
            tf.argmax(logits, 1), tf.argmax(onehot_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        total_batch = int(mnist.train.num_examples // batch_size)
        for epoch in range(20):
            for step in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                _, loss_value = sess.run([train_op, loss],
                                         feed_dict={X: batch_xs, y: batch_ys})
            print('Step: %d, Loss: %f' % (step, loss_value))

        # test
        _a = sess.run(accuracy, feed_dict={
                      X: mnist.test.images, y: mnist.test.labels})
        print('Accuracy: %f' % _a)

実行結果は以下URLで確認できます  
https://console.cloud.google.com/mlengine/jobs
